# parsing the ['authorizedhighways'] column in the all_permits data for OSOW vehicle permits
- ns

- Originally requested by Stephen Yoon  
    - data provided by Stephen's office
- next step ==   
    - get the coordinates for the intersections

In [1]:
# import modules
import pandas as pd
import warnings
import gcsfs
import re

In [2]:
# the original_mapping is needed to standardize the highway names for the various records
original_mapping = {
    "5": "I5", "10": "I10", "15": "I15", "40": "I40", "80": "I80", "105": "I105", "110": "I110",
    "205": "I205", "210": "I210", "215": "I215", "280": "I280", "380": "I380", "405": "I405",
    "505": "I505", "580": "I580", "605": "I605", "680": "I680", "710": "I710", "805": "I805",
    "880": "I880", "980": "I980", "1": "SR1", "2": "SR2", "3": "SR3", "4": "SR4", "7": "SR7",
    "8": "SR8", "9": "SR9", "11": "SR11", "12": "SR12", "13": "SR13", "14": "SR14", "15": "SR15",
    "16": "SR16", "17": "SR17", "18": "SR18", "20": "SR20", "22": "SR22", "23": "SR23",
    "24": "SR24", "25": "SR25", "26": "SR26", "27": "SR27", "28": "SR28", "29": "SR29",
    "32": "SR32", "33": "SR33", "34": "SR34", "35": "SR35", "36": "SR36", "37": "SR37",
    "38": "SR38", "39": "SR39", "41": "SR41", "43": "SR43", "44": "SR44", "45": "SR45",
    "46": "SR46", "47": "SR47", "49": "SR49", "51": "SR51", "52": "SR52", "53": "SR53",
    "54": "SR54", "55": "SR55", "56": "SR56", "57": "SR57", "58": "SR58", "59": "SR59",
    "60": "SR60", "61": "SR61", "62": "SR62", "63": "SR63", "65": "SR65", "66": "SR66",
    "67": "SR67", "68": "SR68", "70": "SR70", "71": "SR71", "72": "SR72", "73": "SR73",
    "74": "SR74", "75": "SR75", "76": "SR76", "77": "SR77", "78": "SR78", "79": "SR79",
    "82": "SR82", "83": "SR83", "84": "SR84", "85": "SR85", "86": "SR86", "87": "SR87",
    "88": "SR88", "89": "SR89", "90": "SR90", "91": "SR91", "92": "SR92", "94": "SR94",
    "96": "SR96", "98": "SR98", "99": "SR99", "103": "SR103", "104": "SR104", "107": "SR107",
    "108": "SR108", "109": "SR109", "110": "SR110", "111": "SR111", "112": "SR112",
    "113": "SR113", "114": "SR114", "115": "SR115", "116": "SR116", "118": "SR118",
    "119": "SR119", "120": "SR120", "121": "SR121", "123": "SR123", "124": "SR124",
    "125": "SR125", "126": "SR126", "127": "SR127", "128": "SR128", "129": "SR129",
    "130": "SR130", "131": "SR131", "132": "SR132", "133": "SR133", "134": "SR134",
    "135": "SR135", "136": "SR136", "137": "SR137", "138": "SR138", "139": "SR139",
    "140": "SR140", "142": "SR142", "144": "SR144", "145": "SR145", "146": "SR146",
    "147": "SR147", "149": "SR149", "150": "SR150", "151": "SR151", "152": "SR152",
    "153": "SR153", "154": "SR154", "155": "SR155", "156": "SR156", "158": "SR158",
    "160": "SR160", "161": "SR161", "162": "SR162", "163": "SR163", "164": "SR164",
    "165": "SR165", "166": "SR166", "167": "SR167", "168": "SR168", "169": "SR169",
    "170": "SR170", "172": "SR172", "173": "SR173", "174": "SR174", "175": "SR175",
    "177": "SR177", "178": "SR178", "180": "SR180", "182": "SR182", "183": "SR183",
    "184": "SR184", "185": "SR185", "186": "SR186", "187": "SR187", "188": "SR188",
    "189": "SR189", "190": "SR190", "191": "SR191", "192": "SR192", "193": "SR193",
    "197": "SR197", "198": "SR198", "200": "SR200", "201": "SR201", "202": "SR202",
    "203": "SR203", "204": "SR204", "207": "SR207", "210": "SR210", "211": "SR211",
    "213": "SR213", "216": "SR216", "217": "SR217", "218": "SR218", "219": "SR219",
    "220": "SR220", "221": "SR221", "222": "SR222", "223": "SR223", "227": "SR227",
    "229": "SR229", "232": "SR232", "233": "SR233", "236": "SR236", "237": "SR237",
    "238": "SR238", "241": "SR241", "242": "SR242", "243": "SR243", "244": "SR244",
    "245": "SR245", "246": "SR246", "247": "SR247", "253": "SR253", "254": "SR254",
    "255": "SR255", "259": "SR259", "260": "SR260", "261": "SR261", "262": "SR262",
    "263": "SR263", "265": "SR265", "266": "SR266", "267": "SR267", "269": "SR269",
    "270": "SR270", "271": "SR271", "273": "SR273", "275": "SR275", "281": "SR281",
    "282": "SR282", "283": "SR283", "284": "SR284", "299": "SR299", "330": "SR330",
    "371": "SR371", "780": "SR780", "905": "SR905", "6": "US6", "50": "US50",
    "95": "US95", "97": "US97", "101": "US101", "199": "US199", "395": "US395"
}

# Generate extended mapping to include leading zeros
road_mapping = {}
for key, value in original_mapping.items():
    road_mapping[key] = value  # Original
    road_mapping[key.zfill(2)] = value  # 2-character zero-padded
    road_mapping[key.zfill(3)] = value  # 3-character zero-padded

#print(road_mapping)

In [3]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/freight/all_permits/"

In [4]:
file_names = ["all_permits_2023_sampleset.xlsx",
              "all_permits_2024_sampleset.xlsx"]

In [5]:
def load_excel_sheets_1(gcs_path, file_names):
    """
    Pull in the first sheet from each Excel file in GCS, add a 'year' column based on the filename,
    and remove records with NaN values in the 'permitnumber' column. Returns a concatenated DataFrame
    with data from all files.

    Parameters:
    gcs_path (str): The Google Cloud Storage path where the files are located.
    file_names (list): A list of Excel file names in the GCS path.

    Returns:
    pd.DataFrame: A single concatenated DataFrame with data from all files, a 'year' column, and
                  records with NaN values in 'permitnumber' removed.
    """
    
    # Create a Google Cloud Storage file system object
    fs = gcsfs.GCSFileSystem()
    
    # List to store all DataFrames
    df_list = []
    
    # Suppress any warnings
    warnings.filterwarnings("ignore")
    
    # Suppress the specific UserWarning
    warnings.filterwarnings(
        "ignore",
        message="Your application has authenticated using end user credentials from Google Cloud SDK without a quota project.",
        category=UserWarning,
        module="google.auth._default"
    )
    
    # Define the columns to keep
    columns_to_keep = ['permitnumber', 'year', 'permitvalidfrom', 'permitvalidto', 
                       'loaddescription', 'origin', 'destination', 'authorizedhighways']
    
    # Loop through each file in the file list
    for file in file_names:
        # Extract the year from the filename
        year = file.split('_')[2]  # Assuming the year is the third element when split by '_'
        
        # Open the file and read only the first sheet
        with fs.open(f"{gcs_path}{file}", 'rb') as f:
            df = pd.read_excel(f, sheet_name=0)  # Load only the first sheet
        
        # Clean headers by removing spaces and making characters lowercase
        df.columns = [col.replace(" ", "").lower() for col in df.columns]
        
        # Add 'year' column
        df['year'] = year
        
        # Filter columns and remove rows with NaN in 'permitnumber'
        df = df[columns_to_keep].dropna(subset=['permitnumber'])
        
        # Append to list
        df_list.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(df_list, ignore_index=True)
    
    return final_df

# Parsing function to create individual route locations
def parse_routes(route_info):
    segments = []

    # Split the data by "from", "to", or standalone dash patterns
    raw_segments = re.split(r'\s*-\s*from\s+|\s*-\s*to\s+|(?<!\s)-\s*', route_info)

    # Process each segment
    for segment in raw_segments:
        # Split by ' - ' or '-'
        sub_segments = re.split(r'\s*-\s*|\s*-\s*', segment)
        
        # Clean and add sub-segments
        segments.extend([sub.strip() for sub in sub_segments if sub.strip()])

    return segments


# Custom parsing function
def extract_location(text):
    # Stop keywords pattern
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway)\b"

    # If 'from' exists, process it as before
    if "from" in text.lower():
        match = re.search(r"from\s+(`.*?`|'.*?'|\w+(?:\s+\w+)*)", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Extract the text after "from"
            # Keep the stop keywords and remove everything after them
            location = re.sub(r"(" + stop_keywords + r").*", r"\1", location, flags=re.IGNORECASE).strip()
            return location
    else:
        # If 'from' doesn't exist, look for a stop keyword and capture location
        match = re.search(r"(`.*?`|'.*?'|\w+(?:\s+\w+)*)\s+(" + stop_keywords + r")", text, re.IGNORECASE)
        if match:
            location = match.group(1)  # Capture location before stop keyword
            return location.strip()

    return None  # If no match is found


# Function to clean each string 
def clean_route(route):
    if not isinstance(route, str):  # Handle non-string entries
        return route
    
    # Define stop keywords regex (including 'exit')
    stop_keywords = r"\b(?:dr|drive|rd|ave|way|pkwy|parkway|skyway|road|avenue|blvd|boulevard|st|street|line|lane|ln|hwy|highway|exit)\b"

    # Extract numeric highway number with optional "rte" or "route" prefixes
    numeric_match = re.match(r"(?:rte|route)?\s*(\d+)", route, flags=re.IGNORECASE)
    if numeric_match:
        return numeric_match.group(1)  # Return the numeric value
    
    # Extract street name up to and including the first stop keyword (but excluding the stop word itself)
    street_match = re.search(rf"(.*?\b{stop_keywords}(?!\s*exit)\b)", route, flags=re.IGNORECASE)
    if street_match:
        return street_match.group(1).strip()  # Return the street name up to the stop keyword (excluding 'exit')
    
    # If neither is found, return the original string
    return route

# A function to parse the ['authorizedhighways'] column to get the route information
def process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping):
    """
    Process a dataframe to format and clean route-related columns.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    parse_routes (function): Function to parse the 'authorizedhighways' column into route segments.
    extract_location (function): Function to extract location from a route segment.
    clean_route (function): Function to clean individual route location entries.
    road_mapping (dict): Mapping dictionary for road numbers to their corresponding classes.

    Returns:
    pandas.DataFrame: The processed dataframe with updated route locations and columns.
    """

    # Format the authorized highways field so the text is not all capitalized
    df['authorizedhighways'] = df['authorizedhighways'].str.capitalize()

    # Apply the parsing function to create lists of individual route locations
    df['route_segments'] = df['authorizedhighways'].apply(parse_routes)

    # Determine the maximum number of locations to create the necessary columns
    max_locations = df['route_segments'].apply(len).max()

    # Create new columns for each route location based on the maximum number of locations
    for i in range(max_locations):
        df[f'route_location_{i}'] = df['route_segments'].apply(lambda x: x[i] if i < len(x) else None)

    # Drop the temporary route_segments column
    df.drop(columns=['route_segments'], inplace=True)

    # Add a new column with all values set to "California"
    df.insert(5, "state", "California")  # Index 5 corresponds to the 6th column position

    # Apply title case to the 'origin' and 'destination' columns
    df['origin'] = df['origin'].str.title()
    df['destination'] = df['destination'].str.title()

    # Apply the extract_location function to the column
    df["route_location_start"] = df["route_location_0"].apply(extract_location)

    # Insert the route_location_start column into the 8th position
    df.insert(8, "route_location_start", df.pop("route_location_start"))

    # Drop the [authorizedhighways] column
    #df.drop(columns=['authorizedhighways'], inplace=True)

    # Drop the route_location_0 field
    df = df.drop(columns=['route_location_0'])

    # Identify target columns excluding "route_location_start"
    route_columns = [col for col in df.columns if col.startswith("route_location_") and col != "route_location_start"]

    # Apply the clean_route cleaning function to the target columns (columns that begin with the words "route_location")
    for col in route_columns:
        df[col] = df[col].apply(clean_route)

    # Iterate through each "route_location_" column to remove the word "exit"
    for col in route_columns:
        df[col] = df[col].apply(lambda x: str(x).replace("exit", "").strip() if isinstance(x, str) else x)

    for col in route_columns:
        df[col] = df[col].astype(str)

    # Update the road numbers to their corresponding road class numbers
    for col in route_columns:
        df[col] = df[col].astype(str).map(road_mapping).fillna(df[col])  # Keep original value if no mapping found

    # Create a new field called 'route_location_origin' that identifies the street and city/state
    df['route_location_origin'] = df['route_location_start'] + " " + df['origin'] + ", " + df['state']

    # Move the new column (['route_location_origin']) to the 9th position
    columns = list(df.columns)
    columns.insert(9, columns.pop(columns.index('route_location_origin')))
    df = df[columns]

    # Remove the 'route_location_destination_city' column if it exists
    if 'route_location_start' in df.columns:
        df = df.drop(columns=['route_location_start'])
    
    return df

# A function to create the ['route_intersection_x'] columns
def process_route_intersections(df):
    """
    Process a dataframe to identify and process route location and intersection columns.
    
    Parameters:
    df (pandas.DataFrame): Input dataframe to process.
    
    Returns:
    pandas.DataFrame: A cleaned dataframe with processed route intersections.
    """

    # Identify all columns with "route_location_" prefix
    route_location_cols = [col for col in df.columns if col.startswith("route_location_")]

    # Initialize a counter for the new intersection column names
    intersection_counter = 0

    # Create new columns for intersections
    for i in range(len(route_location_cols) - 1):
        col1 = route_location_cols[i]
        col2 = route_location_cols[i + 1]

        # Name the new intersection column based on the counter
        intersection_col = f"route_intersection_{intersection_counter}"

        # Combine adjacent columns into one field (handle None gracefully)
        df[intersection_col] = df[col1].astype(str) + " & " + df[col2].astype(str)
        df[intersection_col] = df[intersection_col].replace("None & None", None)  # Optional cleanup for all-None rows
        intersection_counter += 1

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column
    for col in intersection_cols:
        # Replace values ending with " & None" with None (Null)
        df[col] = df[col].apply(lambda x: None if isinstance(x, str) and x.endswith(" & None") else x)

    # Identify columns that start with "route_intersection_"
    intersection_cols = [col for col in df.columns if col.startswith("route_intersection_")]

    # Iterate over each intersection column to remove leading zeros from numeric values
    for col in intersection_cols:
        # Apply the transformation to each value in the column
        df[col] = df[col].apply(lambda x: ' & '.join([part.lstrip('0') if part.isdigit() else part for part in str(x).split(' & ')]) if isinstance(x, str) else x)

    # Create a list(?) called core_columns to be included in the next iteration of the dataframe
    core_columns = [
        "permitnumber", "year", "permitvalidfrom", "permitvalidto",
        "loaddescription", "state", "origin", "destination", "authorizedhighways", "route_location_origin"
    ]

    # subset_columns combines the core columns with the intersection_cols identified earlier in the script
    subset_columns = core_columns + intersection_cols

    # this next line utilizes the defined subset_columns to create a cleaned up version of the dataframe including only the columns needed for this analysis 
    df = df[subset_columns]

    # Replace None values in intersection_cols with empty strings
    for col in intersection_cols:
        df[col] = df[col].apply(lambda x: "" if x is None else x)
    
    return df

# Function to get the last 'route_intersection_x' field
def get_last_intersection(row):
    # Identify columns that match the pattern 'route_intersection_x'
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
    # Get the last non-null value among these columns
    return row[intersection_columns].dropna().iloc[-1] if intersection_columns else None

def add_route_location_destination_city(df):
    # Function to get the last 'route_intersection_x' value
    def get_last_intersection(row):
        # Identify columns that match the pattern 'route_intersection_x'
        intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]
        # Get the last non-null value among these columns
        return row[intersection_columns].dropna().iloc[-1] if len(intersection_columns) > 0 else None

    # Create the new column
    df['route_location_destination_city'] = df.apply(
        lambda row: f"{get_last_intersection(row)} {row['destination']}, {row['state']}", axis=1
    )

    return df

def create_route_intersection_last(df):
    """
    Create a 'route_intersection_last' column to capture the last non-null value
    from all 'route_intersection_x' columns.

    Parameters:
    df (pandas.DataFrame): The dataframe to process.

    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_last' column.
    """
    # Identify all 'route_intersection_x' columns
    intersection_columns = [col for col in df.columns if col.startswith('route_intersection_')]

    if not intersection_columns:
        raise ValueError("No 'route_intersection_' columns found in the dataframe.")

    # Ensure the columns are processed in order
    intersection_columns = sorted(intersection_columns, key=lambda x: int(x.split('_')[-1]))

    # Create 'route_intersection_last' by finding the last non-null value row-wise
    df['route_intersection_last'] = df[intersection_columns].apply(
        lambda row: next((val for val in reversed(row) if pd.notnull(val) and val != ''), None), axis=1
    )

    return df

def create_route_intersection_destination(df):
    """
    Create a new column 'route_intersection_destination' by combining 'route_intersection_last' 
    and 'route_location_destination_city'. Then, clean up the column by removing extra whitespace.
    
    Parameters:
    df (pandas.DataFrame): The input dataframe to process.
    
    Returns:
    pandas.DataFrame: The dataframe with the new 'route_intersection_destination' column and cleaned columns.
    """
    # Create the new 'route_intersection_destination' column
    df['route_intersection_destination'] = df['route_intersection_last'] + " " + df['route_location_destination_city']
    
    # Remove any extra whitespace or empty strings in the resulting combination
    df['route_intersection_destination'] = df['route_intersection_destination'].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # Drop the 'route_location_destination_city' column if it exists
    if 'route_location_destination_city' in df.columns:
        df.drop(columns=['route_location_destination_city'], inplace=True)

    # Drop the 'route_intersection_last' column if it exists
    if 'route_intersection_last' in df.columns:
        df.drop(columns=['route_intersection_last'], inplace=True)

    return df

def process_workflow(gcs_path, file_names):
    # Load the Excel sheets from GCS and get the initial DataFrame
    df = load_excel_sheets_1(gcs_path, file_names)
    
    # Process route locations
    df = process_route_locations(df, parse_routes, extract_location, clean_route, road_mapping)
    
    # Process route intersections
    df = process_route_intersections(df)
    
    # Add route location destination city
    df = add_route_location_destination_city(df)
    
    # Create route intersection last
    df = create_route_intersection_last(df)
    
    # Create route intersection destination
    df = create_route_intersection_destination(df)
    
    # # Replace the last non-null route intersection with destination
    # df = replace_last_non_null_intersection(df)
    
    return df

In [6]:
df = process_workflow(gcs_path, file_names)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
df.head()

,permitnumber,year,permitvalidfrom,permitvalidto,loaddescription,state,origin,destination,authorizedhighways,route_location_origin,...,route_intersection_15,route_intersection_16,route_intersection_17,route_intersection_18,route_intersection_19,route_intersection_20,route_intersection_21,route_intersection_22,route_intersection_23,route_intersection_destination
0,e23-013125,2023,02/15/2023,02/21/2023,75' KELLY BAR,California,Hayward,Antelope,* from clawiter rd s/b on ramp - 092e - 880n -...,"clawiter rd Hayward, California",...,I80 & antelope rd,,,,,,,,,"I80 & antelope rd Antelope, California"
1,e23-021610,2023,03/20/2023,03/26/2023,UNLADEN 9 AXLE WITH 2 DECK INSERTS,California,Fontana,Ontario,* from sierra ave w/b on ramp - 015s - 060w - ...,"sierra ave Fontana, California",...,,,,,,,,,,"SR60 & vineyard ave Ontario, California"
2,e23-022752,2023,03/22/2023,03/28/2023,M95 TRACKED CONVEYOR,California,Dixon,Fresno,* from industrial way - 113n - 080w - 680s - 5...,"industrial way Dixon, California",...,,,,,,,,,,"SR180 & north floyd ave Fresno, California"
3,e23-036568,2023,05/05/2023,05/11/2023,5 TROWEL MACHINES (END TO END) & MISC LEGAL FR...,California,Elk Grove,Ca/Nv Border,* from grant line rd w/b on ramp - 099n - 051n...,"grant line Elk Grove, California",...,,,,,,,,,,"I80 & nv line Ca/Nv Border, California"
4,e23-017248,2023,03/02/2023,03/08/2023,150H GRADER,California,Fairfield,Saratoga,* from air base pkwy n/b on ramp - 080w - 680s...,"air base pkwy Fairfield, California",...,,,,,,,,,,"SR17 & scotts valley dr Saratoga, California"


In [8]:
df.to_csv("justlooking_subset_v1_od.csv", index=False)

In [9]:
# Put this into GitHub, then post in Teams to ask for assistance pulling in the Lat/Lon
# maybe put this on Slack as well

In [10]:
# may have to use mapquest - it was working pretty well to identify the locations